In [1]:
import tensorflow as tf
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import pickle

In [2]:
# load model file and history of the model
saved_dir = 'D:/JATHURSH/Jupyter_notebook_projects/DrowsinessDetectionOnCombinedDataset/saved_data/models/'

model = tf.keras.models.load_model(saved_dir + 'drowsiness_detection_hpt_best_model_2.h5')

with open(saved_dir + 'drowsiness_detection_hpt_best_model_2_history.pkl', 'rb') as file:
    history = pickle.load(file)

In [3]:
# Load the pre-trained Haar Cascade classifier for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Load the pre-trained Haar Cascade classifier for eye detection
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [4]:
# Initialize the video capture from the default camera (0) or a video file
cap = cv2.VideoCapture(0)  # Use 0 for default camera, or provide the video file path

while True:
    # Capture video frame-by-frame
    ret, frame = cap.read()
    
    if not ret:
        break  # Break the loop if no frame is captured
    
    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(10, 10))
    
    for (x, y, w, h) in faces:
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        
        # Crop the face region for eye detection
        roi_gray = gray[y:y + h, x:x + w]
        roi_color = frame[y:y + h, x:x + w]
        
        # Detect eyes in the face region
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=4, minSize=(5, 5))
        
        for (ex, ey, ew, eh) in eyes:
            # Draw a rectangle around the detected eye
            cv2.rectangle(roi_color, (ex, ey), (ex + ew, ey + eh), (255, 0, 0), 2)
            
            # Crop the eye region
            eye_roi = roi_gray[ey:ey + eh, ex:ex + ew]
            
            # Convert the grayscale image to RGB format
            rgb_eye_roi = cv2.cvtColor(eye_roi, cv2.COLOR_GRAY2RGB)
            
            # Resize the eye image to 224x224 pixels
            eye_resized = cv2.resize(rgb_eye_roi, (224, 224))
            
            # Adding the 4th dimension for compatibility with the mobileNetV2 model
            final_eye_image = np.expand_dims(eye_resized, axis=0)
            
            # Normalize - using the same mobileNetV2's preprocess_input method
            final_eye_image_preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(final_eye_image)
            
            # Predict drowsiness on the eye image
            drowsiness_prediction = model.predict(final_eye_image_preprocessed)
            
            # Check if the prediction indicates drowsiness
            is_awake = (drowsiness_prediction >= 0.5).astype(int)
            
            # Display the prediction result on the frame
            label = "Awake" if is_awake else "Drowsy"
            color = (0, 255, 0) if is_awake else (0, 0, 255)
            cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    # Display the frame with detections and predictions
    cv2.imshow("Drowsiness Detection", frame)
    
    # Break the loop if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 85ms/step


1/1 [==============================] - 0s 127ms/step
